In [355]:
import pandas as pd
import sqlite3 as db
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,explained_variance_score
from sklearn.cluster import KMeans
from statsmodels.tsa.seasonal import seasonal_decompose

In [3]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [4]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [5]:
def stopQualityMask(speciesCode: str,closestStop: str):
    cnx = connectDB()
    try:
        gap = dt.date.today().year-2018
        query = f'SELECT COUNT(DISTINCT(year)) as "frq" FROM coefficients_bySpecies WHERE speciesCode = "{speciesCode}" AND closestStop = "{closestStop}"'
        coeficients = pd.read_sql(query,con=cnx)
        coeficients['frq'] = coeficients.apply(lambda g: (g.frq/gap),axis=1)
    except Exception as maskExc:
        raise maskExc
    return coeficients['frq']

In [224]:
def log_stopMetrics(speciesCode,key,testBlob,latestUpdate):
    cnx = connectDB()
    cur = cnx.cursor()
    try:
        sqliteInsert = """INSERT INTO stopMetricsBlob (speciesCode,key,testBlob,latestUpdate) VALUES (?,?,?,?);"""
        logTuple = (speciesCode,key,testBlob,latestUpdate)
        cur.execute(sqliteInsert,logTuple)
        cnx.commit()
        cur.close()
    except db.Error as sqlError:
        raise sqlError
    finally:
        if cnx:
            cnx.close()

In [274]:
#baseline request from the application layer
#outputs a list of birds at the stop and a classification based solely off the number of observations
def birdList_request(StopName: str,cnx):
    try:
        query = f'SELECT speciesCode,count(subId) as "checklists",(SELECT count(subId) FROM historicObservations hxobx WHERE hxobx.speciesCode=hsob.speciesCode) as "sightings" FROM historicObservations hsob LEFT JOIN closestStop on hsob.locId=closestStop.locId WHERE StopName = "{StopName}" GROUP BY speciesCode;'
        sightings = pd.read_sql(sql=query,con=cnx)
    #rareness at the stop
        sightings['stopGroup'] = int
        bucket = sightings['checklists'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['checklists'] <= bucket[0.15],'stopGroup'] = 1  #mythic
        sightings.loc[(sightings['checklists'] > bucket[0.15]) & (sightings['checklists'] <= bucket[0.5]),'stopGroup'] = 2  #rare
        sightings.loc[(sightings['checklists'] > bucket[0.5]) & (sightings['checklists'] < bucket[0.85]),'stopGroup'] = 3   #uncommon
        sightings.loc[(sightings['checklists'] >= bucket[0.85]) & (sightings['checklists'] <=bucket[1]),'stopGroup'] = 4    #common
    #overall rareness
        sightings['overall'] = int
        bucket = sightings['sightings'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['sightings'] <= bucket[0.15],'overall'] = 1
        sightings.loc[(sightings['sightings'] > bucket[0.15]) & (sightings['sightings'] <= bucket[0.5]),'overall'] = 2
        sightings.loc[(sightings['sightings'] > bucket[0.5]) & (sightings['sightings'] < bucket[0.85]),'overall'] = 3
        sightings.loc[(sightings['sightings'] >= bucket[0.85]) & (sightings['sightings'] <=bucket[1]),'overall'] = 4
        sightings['StopName'] = StopName
    #raise an exception if the stopName given is not valid and return a list of valid stop names
    except Exception as ex:
        raise ex
    return sightings


In [275]:
birdlist = birdList_request(StopName='EastSunsetWay',cnx=connectDB())

In [904]:
birdlist[birdlist['speciesCode'] == 'ruckin']


,speciesCode,checklists,sightings,stopGroup,overall,StopName
80,ruckin,28,113,4,3,EastSunsetWay


For birds with robust data, do they behave the same at all stops? If not, what stops does each species appear to prefer? Is that preference explained by habitat?

In [483]:
#TODO #92 change over flat annual average to weekly average in wklyAbd_selectSpecies() and other calcs
def wklyAbd_selectSpecies(cnx,speciesList: list):
    cnx=cnx
    try:
        querySpecies = []
        for i in speciesList:
            i = str(i)
            querySpecies.append(i)
        querySpecies = str(querySpecies).strip('[]')
        #query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.locId in ({query_locIds})'
    #call baseline
        queryBaseline = f'SELECT speciesCode,obsDt,howMany FROM historicObservations WHERE speciesCode in ({querySpecies})'
        baseline = pd.read_sql(sql=queryBaseline,con=cnx,parse_dates=['obsDt'])
        baseline = baseline.assign(obsDt_week=baseline.obsDt.dt.isocalendar().week)
        baseline['howMany'].fillna(1,inplace=True)
        baselineAvg = baseline.groupby(['speciesCode'])['howMany'].mean()

        query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies});'
        obsData = pd.read_sql(query,con=cnx,parse_dates=['obsDt'])
        obsData = obsData.assign(obsDt_week=obsData.obsDt.dt.isocalendar().week)
        obsData['howMany'].fillna(1,inplace=True)
        obsData = obsData.groupby(['speciesCode','StopName','obsDt_week'])['howMany'].mean().reset_index()
    #compute relative abundance
        obsData['relativeAbundance'] = obsData.apply(lambda x: (x.howMany/baselineAvg[x.speciesCode]),axis=1)  #baseline relative abd, same process as Fink et. all
        #obsData['relativeAbundance'] = obsData.apply(lambda f: stopQualityMask(f.speciesCode,f.StopName)*f.relativeAbundance,axis=1)  #apply frequency mask
        #avgAbd = obsData.groupby(['speciesCode'])['relativeAbundance'].mean()
        #obsData['relativeAbundance'] = obsData.apply(lambda n: ((n.relativeAbundance)/(avgAbd[n.speciesCode])),axis=1)  #normalizing around average relative abundance
        obsData.sort_values(by=['StopName','obsDt_week'],ascending=True,inplace=True)
    except db.DatabaseError as dbExc:
        raise f'there was an issue with the database request: {dbExc}'
    except Exception as ex:
        raise ex
    finally: cnx.close()
    return obsData[['obsDt_week','speciesCode','StopName','relativeAbundance']]

Interpolation testing

In [1007]:
ruckin = wklyAbd_selectSpecies(cnx=connectDB(),speciesList=['ruckin'])

In [1097]:
ruckinSchool = ruckin[ruckin['StopName']=='EastSunsetWay']
ruckinSchool

,obsDt_week,speciesCode,StopName,relativeAbundance
4,1,ruckin,EastSunsetWay,0.423221
5,2,ruckin,EastSunsetWay,0.423221
6,3,ruckin,EastSunsetWay,0.423221
7,4,ruckin,EastSunsetWay,0.423221
8,5,ruckin,EastSunsetWay,0.423221
9,6,ruckin,EastSunsetWay,0.423221
10,8,ruckin,EastSunsetWay,0.423221
11,10,ruckin,EastSunsetWay,0.423221
12,14,ruckin,EastSunsetWay,0.423221
13,39,ruckin,EastSunsetWay,0.423221


In [1098]:
#takes a dataframe and returns a dictionary containing the best method and order value
def deriveInterpolationMethod(obsData):
    try:

        testResults = []
        methodDict = [{'method':'linear','order':3,'limit_direction':'both','limit':5},{'method':'slinear','order':3,'limit_direction':'both','limit':5},{'method':'quadratic','order':0,'limit_direction':'both','limit':5},{'method':'cubic','order':3,'limit_direction':'both','limit':5},{'method':'spline','order':3,'limit_direction':'both','limit':5},{'method':'spline','order':5,'limit_direction':'both','limit':5},{'method':'polynomial','order':3,'limit_direction':'both','limit':5},{'method':'polynomial','order':5,'limit_direction':'both','limit':5},{'method':'barycentric','order':5,'limit_direction':'both','limit':5}]
        small_methodDict = [{'method':'linear','order':0,'limit_direction':'both','limit':3},{'method':'slinear','order':3,'limit_direction':'both','limit':3},{'method':'polynomial','order':3,'limit_direction':'both','limit':3},{'method':'quadratic','order':0,'limit_direction':'both','limit':3}]
            
        stop_obsData = obsData
        allweek = pd.DataFrame({'obsDt_week':range(1,53)})
        stop_obsData = pd.merge(left=allweek,left_on='obsDt_week',right=stop_obsData,right_on='obsDt_week',how='left')
    #mask
        stop_obsData['mask'] = stop_obsData['relativeAbundance'].interpolate(method='ffill',limit=4,limit_direction='forward')
        stop_obsData['mask'] = stop_obsData['mask'].interpolate(method='bfill',limit=2,limit_direction='backward')
        stop_obsData.loc[stop_obsData['mask'].isna() == True,'relativeAbundance'] = 0
        stop_obsData = stop_obsData[stop_obsData['relativeAbundance'].notna()].drop(columns=['mask'])
        for v in list([0.2,0.25,0.3,0.35]):
            stop_obsData['sample'] = stop_obsData['relativeAbundance'].sample(frac=v,random_state=1)
            if stop_obsData['speciesCode'].count() < 4:
                chosen = small_methodDict
            else: chosen = methodDict
            blob = {}
            for test in chosen:
                testName = str(test['method'])
                stop_obsData[testName] = stop_obsData['sample']
                stop_obsData[testName] = stop_obsData['sample'].interpolate(method=test['method'],order=test['order'],limit=test['limit'],limit_direction=test['limit_direction']).fillna(stop_obsData['relativeAbundance'])
                r_sqr = r2_score(y_true=stop_obsData['relativeAbundance'],y_pred=stop_obsData[testName])
                if r_sqr < 0:
                    continue
                blob = {'method':test['method'],'order':test['order'],'limit':test['limit'],'sampleSize':v,'r_sqr':r_sqr}
                #blob = {'method':test['method'],'order':test['order'],'sampleSize':v,'r_sqr':r_sqr}
                testResults.append(blob)
        if not testResults:
            testResults.append({'method':'pad','order':0,'limit':0})
            resultFrame = pd.DataFrame.from_dict(testResults)
        else:
            resultFrame = pd.DataFrame.from_dict(testResults)
            resultFrame = resultFrame.groupby(by=['method','order'])['r_sqr'].median().reset_index()
            resultFrame.sort_values(by=['r_sqr'],ascending=False,inplace=True,ignore_index=True)
        #resultFrame.drop_duplicates(subset=['StopName'],keep='first',inplace=True)
                #stop_obsData.rename(columns={'sample':test['method']},inplace=True)
    ##TODO #93 for each stop, return the best interpolation method as determined by the r2 score

    except Exception as problem:
        raise problem
    return resultFrame.loc[0].to_dict()
    #return resultFrame

In [1099]:
ruckin_week = deriveInterpolationMethod(ruckinSchool)
ruckin_week

{'method': 'slinear', 'order': 3, 'r_sqr': 0.9406177020279693}

In [1000]:
##TODO #90 rewrite stopCovariance to run for a single species at a time --DONE

def stopCovariance(obsData: pd.DataFrame,species: str,StopKey: str,cnx):
    try:
        obsData.sort_values(by=['StopName','obsDt_week'],ascending=True,inplace=True)
        container = []
        resultsContainer = []
        stopKeys = obsData.drop_duplicates(subset=['StopName'])
        for StopName in stopKeys.itertuples():
            stop_obsData = obsData[obsData['speciesCode']==species]
            stop_obsData = stop_obsData[stop_obsData['StopName']==StopName.StopName]
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            stop_obsData.drop(columns=['speciesCode','StopName'],inplace=True)
            stop_obsData = pd.merge(left=stop_obsData,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            stop_obsData.set_index('obsDt_week',inplace=True)
            stop_obsData.sort_index(axis='index',ascending=True,inplace=True)
        #TODO #91 write interpolation switches dependent on rarity for stopCovariance()
            stop_obsData['mask'] = stop_obsData['relativeAbundance'].interpolate(method='ffill',limit=5,limit_direction='forward')  #mask, values do not matter
            stop_obsData['mask'] = stop_obsData['mask'].interpolate(method='bfill',limit=2,limit_direction='backward')  #mask, values do not matter
            stop_obsData.loc[stop_obsData['mask'].isna() == True,'relativeAbundance'] = 0
            
            method = deriveInterpolationMethod(stop_obsData)
            stop_obsData['fx_relativeAbundance'] = stop_obsData['relativeAbundance'].interpolate(method=method['method'],order=method['order'],limit_direction='both')



            stop_obsData.drop(columns=['relativeAbundance','mask'],inplace=True)
            stop_obsData.rename(columns={'fx_relativeAbundance':StopName.StopName},inplace=True)              
            container.append(stop_obsData)
        weeklySpeciesAbd = pd.concat(container,ignore_index=False,axis=1)
        weeklySpeciesAbd = weeklySpeciesAbd.fillna(value=0,axis=0)

        #weeklySpeciesAbd = weeklySpeciesAbd.apply(lambda n:(np.log1p(n)),axis=1)
        #y_true = weeklySpeciesAbd[StopKey]
    #return related stops
        #fit = [(stop,r2_score(y_true,weeklySpeciesAbd[stop])) for stop in list(weeklySpeciesAbd)]
        #rSqr = filter(lambda r: (r[1] > 0.25) and (r[1]<1),fit)
        #var = [(stop,explained_variance_score(y_true,weeklySpeciesAbd[stop])) for stop in list(weeklySpeciesAbd)]
        #expVar = filter(lambda e: (e[1]>0.25) and (e[1]<1),var)
        #blob = {'rSquared':list(rSqr),'explVar':list(expVar)}
        #log_stopMetrics(speciesCode=species,key=StopKey,testBlob=str(blob),latestUpdate=str(dt.datetime.today()))

    except Exception as ex:
        raise ex
    return weeklySpeciesAbd
    #return {'speciesCode':species,'stopKey':StopKey,'rSquared':list(rSqr),'explVar':list(expVar)}
    #return method

In [1001]:
stopCovariance(obsData=ruckin,species='ruckin',StopKey='HighSchool',cnx=connectDB())

,ChiricoTrail_PooPooPoint,EastSunsetWay,HighSchool,MargaretsWay,MountTeneriffe,SquakMountain
obsDt_week,,,,,,
1,0.000000,0.423221,0.000000,0.846442,0.000000,0.000000
2,0.000000,0.423221,0.000000,0.846442,0.000000,0.000000
3,0.124663,0.423221,0.423221,0.846442,0.000000,0.423221
4,0.292800,0.423221,-8.562201,0.846442,0.000000,0.846442
5,0.423221,0.423221,-11.014341,0.846442,0.000000,1.058052
6,0.453871,0.423221,-9.658485,1.692884,0.000000,0.846442
7,0.399228,0.423221,-6.517391,1.481273,0.000000,1.185019
8,0.292906,0.423221,-3.001642,1.269663,0.000000,0.987516
9,0.168519,0.423221,0.000000,1.058052,0.000000,0.846442


Habitats

In [1004]:
np.log1p(1.269663)

0.8196313623225296

In [332]:
def kmeans_habitat(cnx,distinctHabitats: int):
    try:
        data = pd.read_sql(sql='SELECT * FROM FAO_by_locId;',con=cnx)
        data = data.drop(columns=['locName']).set_index('locId')
        data.fillna(0,inplace=True)
    #normalize
        maxValue = data.apply(max,axis=1)
        data = data.apply(lambda x: (x/maxValue[x.index]),axis=0)   #min-max normalizing to smooth in proportionality
    #compute kmeans for each locId
        habitat_kmeans = KMeans(n_clusters=distinctHabitats,init='k-means++')
        habitat_kmeans = habitat_kmeans.fit(data.values)
        clusterLabels = habitat_kmeans.labels_
    #define habitats
        habitatFrame = pd.DataFrame(data=clusterLabels,columns=['clusterLabel'],index=data.index).sort_values(by='clusterLabel').reset_index()
        habitatFrame = pd.merge(left=habitatFrame,left_on='locId',right=data,right_on='locId',how='left')
    except Exception as kmeansExc:
        raise kmeansExc
    return habitatFrame     

In [895]:
def wklyAbd_selectSpecies_locIds(cnx,speciesList: list,locIdList: list):
    cnx=cnx
    try:
        querySpecies = []
        for i in speciesList:
            i = str(i)
            querySpecies.append(i)
        querySpecies = str(querySpecies).strip('[]')
        queryLocs = []
        for i in locIdList:
            i = str(i)
            queryLocs.append(i)
        queryLocs = str(queryLocs).strip('[]')
    #baseline
        baseQuery = f'SELECT speciesCode,obsDt,howMany FROM historicObservations WHERE speciesCode in ({querySpecies});'
        baseline = pd.read_sql(baseQuery,con=cnx,parse_dates=['obsDt'])
        baseline = baseline.assign(obsDt_week=baseline.obsDt.dt.isocalendar().week)
        baselineAvg = baseline.groupby(['speciesCode'])['howMany'].mean()
        #query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies}) AND FX.locId IN ({queryLocs});'
        query = f'SELECT speciesCode,FX.locId,obsDt,howMany FROM historicObservations AS FX WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies}) AND FX.locId IN ({queryLocs});'
        obsData = pd.read_sql(query,con=cnx,parse_dates=['obsDt'])
        obsData = obsData.assign(obsDt_week=obsData.obsDt.dt.isocalendar().week)
        obsData['howMany'].fillna(1,inplace=True)
        obsData = obsData.groupby(['speciesCode','obsDt_week'])['howMany'].mean().reset_index()
        #maxCount = obsData.groupby(['speciesCode'])['howMany'].max()
        
    #derive relative abundance
        obsData['relativeAbundance'] = obsData.apply(lambda x: (x.howMany/baselineAvg[x.speciesCode]),axis=1)  #baseline relative abd, same process as Fink et. all
    #obsData['relativeAbundance'] = obsData.apply(lambda f: stopQualityMask(f.speciesCode,f.StopName)*f.relativeAbundance,axis=1)  #apply frequency mask
    #avgAnnualAbd = obsData.groupby(['speciesCode'])['relativeAbundance'].mean()
    #obsData['relativeAbundance'] = obsData.apply(lambda n: ((n.relativeAbundance)/(avgAnnualAbd[n.speciesCode])),axis=1)  #normalizing around average relative abundance
        #obsData.sort_values(by=['obsDt_week'],ascending=True,inplace=True)
    except db.DatabaseError as dbExc:
        raise f'there was an issue with the database request: {dbExc}'
    except Exception as ex:
        raise ex
    finally: cnx.close()
    return obsData

In [384]:
habs = kmeans_habitat(cnx=connectDB(),distinctHabitats=10)
habs.value_counts(subset=['clusterLabel'])

clusterLabel
6               13
0                9
2                8
1                5
8                5
9                5
3                4
7                3
4                2
5                2
dtype: int64

In [893]:
labelList = habs[habs['clusterLabel']==8]
labelZero = list(labelList['locId'])
labelZero

['L8365620', 'L10445835', 'L3438602', 'L1924233', 'L3199734']

In [896]:
result = wklyAbd_selectSpecies_locIds(cnx=connectDB(),speciesList=['gockin'],locIdList=labelZero)
result

,speciesCode,obsDt_week,howMany,relativeAbundance
0,gockin,1,8.0,1.377377
1,gockin,12,8.0,1.377377
2,gockin,14,1.0,0.172172
3,gockin,16,4.0,0.688689
4,gockin,17,5.0,0.860861
5,gockin,18,2.0,0.344344
6,gockin,19,2.0,0.344344
7,gockin,42,1.0,0.172172
8,gockin,50,2.0,0.344344


In [402]:
np.sqrt(((5.837-2.901)**2/3))

1.6951003903407413

In [391]:
gockin = amerob[amerob['speciesCode']=='gockin']

In [544]:
list(amerob[2,])

KeyError: (2,)